In [10]:
import tensorflow as tf
import tensorflow_gnn as tfgnn
from importlib import reload
from sys import modules
from schema import TYPE_SPEC

In [11]:
reload(modules["schema"])
spec = TYPE_SPEC["mnist_graph"]

In [12]:
input = tf.keras.layers.Input(type_spec=spec)
gnn = tfgnn.keras.ConvGNNBuilder(
    lambda edge: tfgnn.keras.layers.SimpleConvolution(
        tf.keras.layers.Dense(32)
    ),
    lambda node: tfgnn.keras.layers.NextStateFromConcat(
        tf.keras.layers.Dense(32),
    ),
    )
hidden = gnn.Convolve()(input)
# hidden = tfgnn.keras.layers.Pool(
#     tag=tfgnn.SOURCE,
#     edge_set_name="connected",
#     reduce_type="sum"
# )(hidden)
hidden = tfgnn.keras.layers.Readout(node_set_name="pixel")(hidden)
hidden = tf.keras.layers.Dense(120, activation="tanh")(hidden)
output = tf.keras.layers.Dense(10, activation="softmax")(hidden)

model = tf.keras.Model(input, output, name = "gnn")
model.summary()

Model: "gnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [()]                      0         
                                                                 
 graph_update_1 (GraphUpdate  ()                       1184      
 )                                                               
                                                                 
 readout_1 (Readout)         (784, 32)                 0         
                                                                 
 dense_2 (Dense)             (784, 120)                3960      
                                                                 
 dense_3 (Dense)             (784, 10)                 1210      
                                                                 
Total params: 6,354
Trainable params: 6,354
Non-trainable params: 0
_____________________________________________________________

In [13]:
model.compile(optimizer="adam", loss=tf.keras.losses.categorical_crossentropy, metrics=["accuracy"])

In [14]:
from generators import load_dataset_from_data
reload(modules["generators"])

(trn_img, trn_lbl), _ = tf.keras.datasets.mnist.load_data()
trn_img = trn_img/255
trn_graph = load_dataset_from_data(trn_img, trn_lbl, 32, spec)

In [15]:
print(type(trn_graph))
print(len(list(trn_graph.take(1))))

<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>
1


2022-03-01 21:28:52.081571: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [ ]:
model.fit(trn_graph, epochs=10)